In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
# Load libraries
from sklearn.dummy import DummyClassifier
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics 
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge

In [2]:
# Read the CSV file using Pandas.
alldata = pd.read_csv('sberbank.csv')

# Convert the timestamp string to an integer representing the year.
def get_year(timestamp):
    return int(timestamp[:4])
alldata['year'] = alldata.timestamp.apply(get_year)

# Select the 9 input columns and the output column.
selected_columns = ['price_doc', 'year', 'full_sq', 'life_sq', 'floor', 'num_room', 'kitch_sq', 'full_all']
alldata = alldata[selected_columns]
alldata = alldata.dropna()

# Shuffle.
alldata_shuffled = alldata.sample(frac=1.0, random_state=0)

# Separate the input and output columns.
X = alldata_shuffled.drop('price_doc', axis=1)
# For the output, we'll use the log of the sales price.
Y = alldata_shuffled['price_doc'].apply(np.log)

# Split into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

In [3]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate
m1 = DummyRegressor()
cross_validate(m1, Xtrain, Ytrain, scoring='neg_mean_squared_error')

{'fit_time': array([0.0045619 , 0.00299764, 0.00299978, 0.00305963, 0.00299835]),
 'score_time': array([0.0010035 , 0.00099969, 0.00099945, 0.        , 0.        ]),
 'test_score': array([-0.39897319, -0.37113485, -0.38083108, -0.39057156, -0.40475168])}

### GradientBoostingRegressor

In [4]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [5]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [6]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}


In [7]:
rf_Model = GradientBoostingRegressor()

In [8]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [9]:
rf_Grid.fit(Xtrain, Ytrain)

Fitting 3 folds for each of 160 candidates, totalling 480 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=4,
             param_grid={'max_depth': [2, 4], 'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72,
                                          80]},
             verbose=2)

In [79]:
rf_Grid.best_params_
print (f'Train Accuracy - : {rf_Grid.score(Xtrain,Ytrain):.3f}')
print (f'Test Accuracy - : {rf_Grid.score(Xtest,Ytest):.3f}')

Train Accuracy - : 0.369
Test Accuracy - : 0.335


Description:
Gradient Boosting regression (GB) is a model which is used to handle problems like classification and regression respectively. GB uses the model with a forward stage-wise, by doing this the model can allow for optimization of the loss funtion which contains arbritaty differentiable. 

From the Gradient Boosting Regressor we got a train accuracy of: 0.369, and a test accuracy of 0.335. 

The hyper parameters used were the following:
- learning_rate: the learning rate is compressing the share for each tree. The used learning rates is [ 0.25 , 0.3 , 0.35, 0.4,0.5 ]
- max_features: number of features to take in to acount when trying to find the best split of n_features. There are three different methods, but in the test we only used two of them, ['auto', 'sqrt']. 
    - Auto: takes all of n features as a parameter
    - Sqrt: takes the square root of the n parameters aa the parameter
    - log2: 
- max_depth: max depth limits the amount of nodes in the regression tree. Different tested depth [ 1, 2, 3, 4, 5,6,7,8,9,10]
- min_samples_split: min samples split is the minimum amount of a sample to be splitted into a node. Tested min sample split as hyperparameters [0.001, 0.5, 10]
- min_samples_leaf: min samples leaf is teh minimum amount of a sample to create a leaf node. Tested min sample leaf [0.001, 0.03, 5]

In [ ]:
   #"loss":["deviance"],
    "learning_rate": [ 0.25 , 0.3 , 0.35, 0.4,0.5 ], #0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2 , 0.6 ,0.7 , 0.8 ,0.9
    "min_samples_split": np.linspace(0.001, 0.5, 10),
    #"n_estimators": [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)],
    "min_samples_leaf": np.linspace(0.001, 0.03, 5),
    "max_depth":[ 1, 2, 3, 4, 5,6,7,8,9,10], #1, 2, 3, 4, ,15,20,30,40,50
    "max_features":["log2","sqrt",'auto'], #1,2,3, ,5,7,9,11,15,17,21
    "criterion": ["friedman_mse",  "mse"],
    #"bootstrap": [True, False]

### Linear regression

In [20]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
lr_s = GridSearchCV(estimator = regressor, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)
regressor.fit(Xtrain, Ytrain)

LinearRegression()

In [21]:
print(regressor.intercept_)
print(regressor.coef_)

-94.93969323686758
[ 5.45064548e-02  1.14202154e-02 -8.44530666e-05  8.31714263e-03
  8.99332453e-02 -1.49600124e-04  4.32154471e-08]


In [22]:
y_pred = regressor.predict(Xtest)

In [23]:
df = pd.DataFrame({'Actual': Ytest, 'Predicted': y_pred})
df

,Actual,Predicted
24987,15.598902,15.378263
12766,15.464169,15.296538
22648,15.789592,15.576978
12435,15.761421,15.596663
24685,13.815511,15.287299
...,...,...
20047,16.320628,16.077809
28460,15.788202,15.491603
26269,15.830414,15.543966
24955,14.508658,15.423333


In [16]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(Ytest, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Ytest, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Ytest, y_pred)))

Mean Absolute Error: 0.3819502113302777
Mean Squared Error: 0.3155890397003741
Root Mean Squared Error: 0.5617731211978498


In [27]:
df['Predicted']

24987    15.378263
12766    15.296538
22648    15.576978
12435    15.596663
24685    15.287299
           ...    
20047    16.077809
28460    15.491603
26269    15.543966
24955    15.423333
14305    15.887954
Name: Predicted, Length: 3352, dtype: float64

### DecisionTreeRegressor

In [70]:
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

def fit_model(X, y):
    regressor = DecisionTreeRegressor()
    
    # test parameters for the depth
    parameters = {'max_depth':(1,2,3,4,5,6,7,8,9,10),
              'max_features' : ('auto', 'sqrt', 'log2')
              , 'min_samples_split' : (2,4,6,8,10)}
    scoring_function = make_scorer(fbeta_score, beta=2)
    reg = GridSearchCV(regressor, param_grid=parameters, scoring=scoring_function)
    print(reg)
    reg.fit(X, y)
    return reg.best_estimator_

reg = fit_model(Xtrain, Ytrain)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
                         'max_features': ('auto', 'sqrt', 'log2'),
                         'min_samples_split': (2, 4, 6, 8, 10)},
             scoring=make_scorer(fbeta_score, beta=2))


In [71]:
reg

DecisionTreeRegressor(max_depth=1, max_features='auto')

In [80]:
print (f'Train Accuracy - : {reg.score(Xtrain,Ytrain):.3f}')
print (f'Test Accuracy - : {reg.score(Xtest,Ytest):.3f}')

Train Accuracy - : 0.161
Test Accuracy - : 0.153


Description:
Decision Tree Regressor/classification (DTR) models have the structure as a tree. The DTR takes in a data set and splits it down to smaller chunks. This leaves the tree structure with decision nodes and leaf nodes respectivly. A node in this case can have two branches as a minimum, this nodes is a representation of teh tested values. The leaf nodes is representing the the actual decision.

From the Decision Tree Regressor we got a train accuracy of: 0.161, and a test accuracy of 0.153.

Used hyper parameters:
- max_depth: maximum depth of the tree. Here we used a series of different max depths (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
- max_features: max features has three different ways to consider the best split (auto, sqrt, log2). 
- min_samples_leaf: min samples leaf is teh minimum amount of a sample to create a leaf node. Used range [0.001, 0.03, 5] 
- random_state: random sate is a random control that controls the randomly permuted split. It choose either max_feature or n_feature, depending which of them are smallest. The best split is when these two para meters are equal to each otehr. We used the the following random states [0, 1, 2, 3, 4, 5] 


In [ ]:
'max_depth': [1, 2, 3, 4, 5],  #6,7,8,9,10,15,20,30,40,50
                  'max_features': [1, 2, 3, 4,5,], #6,7,8,9,10,12,14,16,18,19,21
                 'random_state':[0, 1, 2, 3, 4, 5],  #10, 15,20,35,50,80,100,150,180,200],
                  #'min_samples_split':[1, 2, 5, 7, 10, 15, 20, 30],
                  "min_samples_leaf": np.linspace(0.001, 0.03, 5)
                 #'criterion':['gini','entropy'],

### Random Forest Regressor

In [74]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [110]:
regr = RandomForestRegressor(n_estimators=100,   
                             max_depth=None, 
                             min_samples_split=2, 
                             min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0, 
                             max_features='sqrt', max_leaf_nodes=None, 
                             min_impurity_decrease=0.0, bootstrap=True, 
                             oob_score=False, n_jobs=None, random_state=None, 
                             verbose=0, warm_start=False, ccp_alpha=0.0, 
                             max_samples=None)
regr.fit(Xtrain, Ytrain)

RandomForestRegressor(max_features='sqrt')

In [111]:
print (f'Train Accuracy - : {regr.score(Xtrain,Ytrain):.3f}')
print (f'Test Accuracy - : {regr.score(Xtest,Ytest):.3f}')

Train Accuracy - : 0.891
Test Accuracy - : 0.276


Description:
Random Forest Regressor (RF) works as an estimator to classify numbers that belongs to the decision tree, which contains sampels from the dataset that was provided. RF uses an averageing technique to make its predictions and accuracy. There are also an over-fitting control. The RF model uses the whole dataset to construct the tree.

Used hyper parameters:
- max_depth: maximum depth of the tree. Here we used a series of different max depths [1, 2, 3, 4,5,6,7, 8,9,10,15,20,30,40,50]
- min_samples_split: min samples split is the minimum amount of a sample to be splitted into a node. Used values [0.001, 0.5, 10]
- min_samples_leaf: min samples leaf is teh minimum amount of a sample to create a leaf node. Used hyper parameters [0.001, 0.5, 10]
- max_features: max features has three different ways to consider the best split. Used features as hyper parameters [log2,sqrt]

### References
- https://www.saedsayad.com/decision_tree_reg.htm [DecisionTreeRegressor]

In [ ]:
    'max_depth': [1, 2, 3, 4,5,6,7, 8,9,10,15,20,30,40,50],
     "min_samples_split": np.linspace(0.001, 0.5, 10),
     "min_samples_leaf": np.linspace(0.001, 0.5, 10),
     #'criterion':['auto','gini','entropy'],
    "max_features":["log2","sqrt"],